In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
import phase_diagram as funcs

lead_pars = dict(
    a=10, r1=50, r2=70, coverage_angle=135, angle=45, with_shell=True, which_lead=""
)

params = dict(
    alpha=20,
    B_x=0,
    B_y=0,
    B_z=0,
    Delta=110,
    g=50,
    orbital=True,
    mu_sc=100,
    c_tunnel=3 / 4,
    V_r=-50,
    intrinsic_sc=False,
    mu_=lambda x0, sigma, mu_lead, mu_wire: mu_lead,
    V_=lambda z, V_0, V_r, V_l, x0, sigma, r1: 0,
    V_0=None,
    V_l=None,
    mu_lead=10,
    mu_wire=None,
    r1=None,
    sigma=None,
    x0=None,
    **funcs.constants.__dict__
)


def pf(xy, params=params, lead_pars=lead_pars):
    import phase_diagram as funcs

    params["B_x"], params["mu_lead"] = xy
    lead = funcs.make_lead(**lead_pars).finalized()
    return funcs.calculate_pfaffian(lead, params)


def smallest_gap(xy, params=params, lead_pars=lead_pars):
    import phase_diagram as funcs

    params["B_x"], params["mu_lead"] = xy
    lead = funcs.make_lead(**lead_pars).finalized()
    pf = funcs.calculate_pfaffian(lead, params)
    gap = funcs.gap_from_modes(lead, params)
    return pf * gap

In [ ]:
learner = adaptive.Learner2D(smallest_gap, bounds=[(0, 2), (0, 35)])
# learner.load('phase_diagram.pickle')

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.npoints > 60000)

In [ ]:
runner.live_info()

In [ ]:
%%output size=100
learner.plot(tri_alpha=0.4, n=None)

In [ ]:
learner.save('phase_diagram.pickle')